In [19]:
from authenticate import authenticate_google_calendar 

In [20]:
import datetime as dt

In [21]:

def fetch_study_events(service, calendar_id="primary"):
    now = dt.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    # Calculate the start of the month
    start_of_month = (
        dt.datetime.utcnow().replace(day=1) - datetime.timedelta(days=1)
    ).replace(day=1).isoformat() + "Z"
    print("Fetching past month events")
    events_result = (
        service.events()
        .list(
            calendarId=calendar_id,
            timeMin=start_of_month,
            timeMax=now,
            maxResults=100,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
        print("No past month events found.")
        return []

    for event in events:
        start = event["start"].get("dateTime", event["start"].get("date"))
        print(start, event["summary"])

    return events



In [22]:
import datetime as dt
import pytz

def create_study_plan(service, subjects, num_days, block_length=40, calendar_id="primary"):
    START_TIME = dt.time(9, 0)
    END_TIME = dt.time(21, 30)
    LUNCH_DURATION = 45  # Duration of lunch break in minutes
    
    def delete_overlapping_events(start_time, end_time):
        """Deletes events in the study plan that overlap with the given time range."""
        events_result = service.events().list(
            calendarId=calendar_id,
            timeMin=start_time.isoformat(),
            timeMax=end_time.isoformat(),
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])

        for event in events:
            if 'summary' in event and event['summary'] in subjects:
                service.events().delete(calendarId=calendar_id, eventId=event['id']).execute()
                print(f"Deleted overlapping event: {event['summary']} from {event['start']['dateTime']} to {event['end']['dateTime']}.")

    now = dt.datetime.now(pytz.timezone('Europe/London'))
    current_date = now.date()
    study_dates = [current_date + dt.timedelta(days=i) for i in range(num_days)]
    
    total_weight = sum(subjects.values())

    for study_date in study_dates:
        if study_date == current_date and now.time() > START_TIME:
            start_time = max(now, pytz.timezone('Europe/London').localize(dt.datetime.combine(study_date, START_TIME)))
        else:
            start_time = pytz.timezone('Europe/London').localize(dt.datetime.combine(study_date, START_TIME))

        end_time = pytz.timezone('Europe/London').localize(dt.datetime.combine(study_date, END_TIME))
        lunch_start = pytz.timezone('Europe/London').localize(dt.datetime.combine(study_date, dt.time(12, 0)))
        lunch_end = lunch_start + dt.timedelta(minutes=LUNCH_DURATION)

        # Delete overlapping events in the study plan
        delete_overlapping_events(start_time, end_time)

        # Calculate the total study time available
        time_available = end_time - max(start_time, now)

        # Calculate the number of study blocks available based on the specified block length
        num_blocks_available = int(time_available.total_seconds() / (block_length * 60))
        
        # Check if lunch is within the study time and account for it
        if lunch_start > now:
            num_blocks_available -= 1

        # Distribute blocks based on the subject weight
        subject_blocks = {subject: int(weight / total_weight * num_blocks_available) for subject, weight in subjects.items()}
        
        remaining_blocks = num_blocks_available - sum(subject_blocks.values())
        
        # Assign remaining blocks to the subject with the highest weight
        if remaining_blocks > 0:
            max_weight_subject = max(subjects, key=subjects.get)
            subject_blocks[max_weight_subject] += remaining_blocks

        # Output the number of blocks per subject
        print("Number of blocks per subject:")
        for subject, blocks in subject_blocks.items():
            print(f"{subject}: {blocks} blocks")
        
        # Insert events for each subject based on their allocated blocks
        for subject, blocks in subject_blocks.items():
            for _ in range(blocks):
                if start_time >= end_time:
                    break
                
                # Skip inserting block during lunch time
                if lunch_start <= start_time < lunch_end:
                    start_time = lunch_end
                    continue
                
                event_end = start_time + dt.timedelta(minutes=block_length)
                if event_end > end_time:
                    break

                event = {
                    "summary": subject,
                    "description": "Study block generated by study planner - Bill Rimell.",
                    "start": {
                        "dateTime": start_time.isoformat(),
                        "timeZone": "Europe/London",
                    },
                    "end": {
                        "dateTime": event_end.isoformat(),
                        "timeZone": "Europe/London",
                    },
                }
                service.events().insert(calendarId=calendar_id, body=event).execute()
                print(f"Added {subject} study block from {start_time} to {event_end}.")
                
                # 5-minute break between study blocks
                start_time += dt.timedelta(minutes=block_length + 5)

        # Add lunch event if it's within the study time
        if lunch_start < end_time and lunch_end > start_time:
            lunch_end = min(lunch_end, end_time)
            lunch_event = {
                "summary": "Lunch",
                "description": "Lunch break generated by study planner - Bill Rimell.",
                    "start": {
                        "dateTime": lunch_start.isoformat(),
                        "timeZone": "Europe/London",
                    },
                    "end": {
                        "dateTime": lunch_end.isoformat(),
                        "timeZone": "Europe/London",
                    },
                }
            service.events().insert(calendarId=calendar_id, body=lunch_event).execute()
            print(f"Added lunch break from {lunch_start} to {lunch_end}.")


In [23]:
calendar_id = "0ca09266015f691eebe0d00c6f3ed7a784713e0160a694b8f7929add00cb1aa1@group.calendar.google.com"

In [24]:
service = authenticate_google_calendar()

subjects = {'Maths': 150, 'Physics': 0, 'Computer Science': 0}  # Example subjects with weights
create_study_plan(service=service, subjects=subjects, num_days=2, block_length=120, calendar_id=calendar_id)  # Specify your calendar ID


Deleted overlapping event: Maths from 2024-04-27T11:45:00Z to 2024-04-27T12:25:00Z.
Deleted overlapping event: Maths from 2024-04-27T12:30:00Z to 2024-04-27T13:10:00Z.
Deleted overlapping event: Maths from 2024-04-27T13:15:00Z to 2024-04-27T13:55:00Z.
Deleted overlapping event: Maths from 2024-04-27T14:00:00Z to 2024-04-27T14:40:00Z.
Deleted overlapping event: Maths from 2024-04-27T14:45:00Z to 2024-04-27T15:25:00Z.
Deleted overlapping event: Maths from 2024-04-27T15:30:00Z to 2024-04-27T16:10:00Z.
Deleted overlapping event: Maths from 2024-04-27T16:15:00Z to 2024-04-27T16:55:00Z.
Deleted overlapping event: Maths from 2024-04-27T17:00:00Z to 2024-04-27T17:40:00Z.
Deleted overlapping event: Maths from 2024-04-27T17:45:00Z to 2024-04-27T18:25:00Z.
Deleted overlapping event: Maths from 2024-04-27T18:30:00Z to 2024-04-27T19:10:00Z.
Deleted overlapping event: Maths from 2024-04-27T19:15:00Z to 2024-04-27T19:55:00Z.
Number of blocks per subject:
Maths: 4 blocks
Physics: 0 blocks
Computer Sci